In [3]:
pip install dash

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/00/69/d0ee9c6524678e98d336464d5c898182794c6b2ba1e5507bc7010a126ce2/dash-2.14.1-py3-none-any.whl.metadata
  Using cached dash-2.14.1-py3-none-any.whl.metadata (11 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Using cached retrying-1.3.4-py3-none-any.whl (11 kB)
  Using cached ansi2html-1.8.0-py3-none-any.whl (16 kB)
Using cached dash-2.14.1-py3-none-any.whl (10.4 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kern

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.io as pio
from dash import Dash, dcc, html
from datetime import datetime, timedelta
import numpy as np

In [4]:
stock_data = pd.read_csv("preprocessed_CAC40.csv")
stock_data['Volume'] = pd.to_numeric(stock_data['Volume'].str.replace(',', ''), errors='coerce')

print("Imported file preprocessed_CAC40.csv successfully")

stock_data

Imported file preprocessed_CAC40.csv successfully


,Unnamed: 0,Name,Date,Open,Closing_Price,Daily_High,Daily_Low,Volume
0,0,Accor,2020-04-03,22.99,23.40,23.40,22.99,67.0
1,1,Accor,2020-04-02,23.91,22.99,23.91,22.99,250.0
2,2,Accor,2020-04-01,24.10,23.83,24.10,23.83,37.0
3,3,Accor,2020-03-31,25.04,25.00,25.24,24.99,336.0
4,4,Accor,2020-03-30,26.50,25.02,26.50,24.99,415.0
...,...,...,...,...,...,...,...,...
97643,97643,Worldline SA,2014-07-04,16.22,16.28,16.29,16.22,NaN
97644,97644,Worldline SA,2014-07-03,16.12,16.28,16.28,16.12,NaN
97645,97645,Worldline SA,2014-07-02,16.22,16.17,16.32,16.17,NaN
97646,97646,Worldline SA,2014-07-01,16.42,16.25,16.55,16.25,NaN


In [6]:
# Ensuring that the time filter uses the must curent date in the dataset 
current_time_str = stock_data['Date'].max()
current_time = datetime.strptime(current_time_str, '%Y-%m-%d')
print(current_time)

2020-04-03 00:00:00


In [ ]:
# Creating an interactive line graph for closing prices for each company
# Convert 'Date' column to datetime format
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Create a dropdown filter for company selection
companies = stock_data['Name'].unique().tolist()

# Create an interactive time-series line chart using Plotly Express
fig = px.line(title='Interactive Time-Series Line Chart For Stock Closing Prices')

# Iterate through each company to add traces to the figure
for company in companies:
    company_data = stock_data[stock_data['Name'] == company]
    fig.add_scatter(x=company_data['Date'], y=company_data['Closing_Price'], name=company, visible=False)

# Set the visibility of the first company's trace to True initially
fig.data[0].visible = True

# Create buttons for dropdown selection
buttons = []
for i, company in enumerate(companies):
    visibility = [i == j for j in range(len(companies))]
    button = dict(label=company, method="update", args=[{"visible": visibility}, {"title": f"Interactive Time-Series Line Chart - {company} Closing Prices"}])
    buttons.append(button)

# Update layout to include dropdown selection
fig.update_layout(
    updatemenus=[{"buttons": buttons, "direction": "down", "showactive": True, "x": 0.5, "y": 1.5}],
    xaxis_title='Date',
    yaxis_title='Closing Stock Price'
)

# Show the chart
fig.show()

In [ ]:
# Creating an interactive candlestick for all companies
# Create a dropdown filter for company selection
companies = stock_data['Name'].unique().tolist()

# Create an empty figure
fig = go.Figure()

# Iterate through each company to add candlestick traces to the figure
for company in companies:
    company_data = stock_data[stock_data['Name'] == company]
    fig.add_trace(go.Candlestick(x=company_data['Date'],
                                 open=company_data['Open'],
                                 high=company_data['Daily_High'],
                                 low=company_data['Daily_Low'],
                                 close=company_data['Closing_Price'],
                                 name=company,
                                 visible=False))

# Set the visibility of the first company's trace to True initially
fig.data[0].visible = True

# Create buttons for dropdown selection
buttons = []
for i, company in enumerate(companies):
    visibility = [i == j for j in range(len(companies))]
    button = dict(label=company, method="update", args=[{"visible": visibility}, {"title": f"Interactive Candlestick Chart - {company}"}])
    buttons.append(button)

# Update layout to include dropdown selection
fig.update_layout(
    updatemenus=[{"buttons": buttons, "direction": "down", "showactive": True, "x": 0.5, "y": 1.5}],
    xaxis_title='Date',
    yaxis_title='Price',
    title='Interactive Candlestick Chart - Closing Prices'
)

# Show the chart
fig.show()

In [ ]:
# Calculate VWAP for each company
stock_data['VWAP'] = (stock_data['Closing_Price'] * stock_data['Volume']).cumsum() / stock_data['Volume'].cumsum()

# Create Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='company-dropdown',
        options=[{'label': company, 'value': company} for company in stock_data['Name'].unique()],
        value=stock_data['Name'].iloc[0]  # Default value for dropdown
    ),
    dcc.Graph(id='vwap-chart')
])

# Define callback to update the VWAP chart based on selected company
@app.callback(
    Output('vwap-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_chart(selected_company):
    filtered_stock_data = stock_data[stock_data['Name'] == selected_company]
    fig = px.scatter(filtered_stock_data, x='Date', y='VWAP', title=f'Volume-Weighted Average Price (VWAP) for {selected_company}',
                     color='Closing_Price',  # Encode Volume information using color
                     size='Closing_Price',   # Encode Volume information using marker size
                     color_continuous_scale='Viridis',  # Choose a color scale
                     labels={'VWAP': 'VWAP', 'Closing_Price': 'Closing_Price'},  # Set labels for axes
                     hover_data={'Date': '|%B %d, %Y'})  # Format hover data for date
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, port = 8050)

In [ ]:
# Calculate VWAP for each company
stock_data['VWAP'] = (stock_data['Closing_Price'] * stock_data['Volume']).cumsum() / stock_data['Volume'].cumsum()

# Create Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.Label('Select Company:'),
    dcc.Dropdown(
        id='company-dropdown',
        options=[{'label': company, 'value': company} for company in stock_data['Name'].unique()],
        value=stock_data['Name'].iloc[0]  # Default value for dropdown
    ),
    html.Br(),
    html.Label('Select Timeframe:'),
    dcc.RadioItems(
        id='timeframe-radio',
        options=[
            {'label': 'Last 1 Month', 'value': '1M'},
            {'label': 'Last 5 Months', 'value': '5M'},
            {'label': 'Last 1 Year', 'value': '1Y'},
            {'label': 'Last 5 Years', 'value': '5Y'},
            {'label': 'Entire Duration', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'block'}
    ),
    dcc.Graph(id='vwap-chart')
])

# Define callback to update the VWAP chart based on selected company and timeframe
@app.callback(
    Output('vwap-chart', 'figure'),
    [Input('company-dropdown', 'value'),
     Input('timeframe-radio', 'value')]
)
def update_chart(selected_company, selected_timeframe):
    filtered_stock_data = stock_data[stock_data['Name'] == selected_company]

    if selected_timeframe == '1M':
        start_date = current_time - timedelta(days=30)
    elif selected_timeframe == '5M':
        start_date = current_time - timedelta(days=150)
    elif selected_timeframe == '1Y':
        start_date = current_time - timedelta(days=365)
    elif selected_timeframe == '5Y':
        start_date = current_time - timedelta(days=1825)  # Approximation of 5 years
    else:
        start_date = filtered_stock_data['Date'].min()  # Entire duration

    filtered_stock_data = filtered_stock_data[pd.to_datetime(filtered_stock_data['Date']) >= start_date]

    fig = px.line(filtered_stock_data, x='Date', y='VWAP',
                  title=f'Volume-Weighted Average Price (VWAP) for {selected_company}',
                  line_shape='linear')  # Set line_shape to linear for continuous lines

    fig.add_scatter(x=filtered_stock_data['Date'], y=filtered_stock_data['Closing_Price'],
                    mode='markers',
                    marker=dict(color=filtered_stock_data['Closing_Price'],
                                colorscale='Viridis', opacity=0.7, colorbar=dict(title='Closing Price'),
                                line=dict(width=1, color='DarkSlateGrey')),
                    name='Price vs Volume')

    fig.update_traces(line=dict(width=1), selector=dict(type='scatter'))  # Increase line width for main plot lines

    fig.update_layout(
        xaxis=dict(title='Date', tickformat='%Y-%m-%d', showgrid=True, linecolor='black', linewidth=1),
        yaxis=dict(title='VWAP', showgrid=True, linecolor='black', linewidth=1),
        paper_bgcolor='white',  # Set paper (outside plot area) background color
        font=dict(family='Arial', size=10, color='Black')  # Set font style and size
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, port = 8051)

In [ ]:
# Calculate Simple Moving Average (SMA) for Closing_Price (taking a window of 3 days as an example)
stock_data['SMA'] = stock_data['Closing_Price'].rolling(window=3).mean()

# Create Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.Label('Select Company:'),
    dcc.Dropdown(
        id='company-dropdown',
        options=[{'label': company, 'value': company} for company in stock_data['Name'].unique()],
        value=stock_data['Name'].iloc[0]  # Default value for dropdown
    ),
    html.Br(),
    html.Label('Select Timeframe:'),
    dcc.RadioItems(
        id='timeframe-radio',
        options=[
            {'label': 'Last 1 Month', 'value': '1M'},
            {'label': 'Last 5 Months', 'value': '5M'},
            {'label': 'Last 1 Year', 'value': '1Y'},
            {'label': 'Last 5 Years', 'value': '5Y'},
            {'label': 'Entire Duration', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'block'}
    ),
    dcc.Graph(id='sma-chart')
])

# Define callback to update the SMA chart based on selected company and timeframe
@app.callback(
    Output('sma-chart', 'figure'),
    [Input('company-dropdown', 'value'),
     Input('timeframe-radio', 'value')]
)
def update_chart(selected_company, selected_timeframe):
    filtered_stock_data = stock_data[stock_data['Name'] == selected_company]

    if selected_timeframe == '1M':
        start_date = current_time - timedelta(days=30)
    elif selected_timeframe == '5M':
        start_date = current_time - timedelta(days=150)
    elif selected_timeframe == '1Y':
        start_date = current_time - timedelta(days=365)
    elif selected_timeframe == '5Y':
        start_date = current_time - timedelta(days=1825)  # Approximation of 5 years
    else:
        start_date = filtered_stock_data['Date'].min()  # Entire duration

    filtered_stock_data = filtered_stock_data[pd.to_datetime(filtered_stock_data['Date']) >= start_date]

    fig = px.line(filtered_stock_data, x='Date', y=['Closing_Price', 'SMA'],
                  title=f'Simple Moving Average (SMA) for {selected_company}')
    
    # Handle NaN values in 'SMA' column by replacing them with a default size
    sizes = filtered_stock_data['SMA'].fillna(10)  # Replace NaN values with default size 10
    
    # Add scatter plot for markers and channels
    fig.add_scatter(x=filtered_stock_data['Date'], y=filtered_stock_data['Closing_Price'],
                    mode='markers',
                    marker=dict(size=sizes, color=filtered_stock_data['SMA'],
                                colorscale='Viridis', opacity=0.7, colorbar=dict(title='SMA'),
                                line=dict(width=1, color='DarkSlateGrey')),
                    name='Price vs SMA')

    fig.update_traces(mode='lines', selector=dict(type='scatter'))  # Set lines for main plot lines

    fig.update_layout(
        xaxis=dict(title='Date', tickformat='%Y-%m-%d', showgrid=True, linecolor='black', linewidth=1),
        yaxis=dict(title='Price', showgrid=True, linecolor='black', linewidth=1),
        paper_bgcolor='white',  # Set paper (outside plot area) background color
        font=dict(family='Arial', size=12, color='Black')  # Set font style and size
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, port = 8052)

In [ ]:
# Calculate RSI for Closing_Price (taking a window of 14 days as an example)
delta = stock_data['Closing_Price'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
rsi = 100 - (100 / (1 + rs))

# Create Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.Label('Select Company:'),
    dcc.Dropdown(
        id='company-dropdown',
        options=[{'label': company, 'value': company} for company in stock_data['Name'].unique()],
        value=stock_data['Name'].iloc[0]  # Default value for dropdown
    ),
    html.Br(),
    html.Label('Select Timeframe:'),
    dcc.RadioItems(
        id='timeframe-radio',
        options=[
            {'label': 'Last 1 Month', 'value': '1M'},
            {'label': 'Last 5 Months', 'value': '5M'},
            {'label': 'Last 1 Year', 'value': '1Y'},
            {'label': 'Last 5 Years', 'value': '5Y'},
            {'label': 'Entire Duration', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'block'}
    ),
    dcc.Graph(id='rsi-chart')
])

# Define callback to update the RSI chart based on selected company and timeframe
@app.callback(
    Output('rsi-chart', 'figure'),
    [Input('company-dropdown', 'value'),
     Input('timeframe-radio', 'value')]
)
def update_chart(selected_company, selected_timeframe):
    filtered_stock_data = stock_data[stock_data['Name'] == selected_company]

    if selected_timeframe == '1M':
        start_date = current_time - timedelta(days=30)
    elif selected_timeframe == '5M':
        start_date = current_time - timedelta(days=150)
    elif selected_timeframe == '1Y':
        start_date = current_time - timedelta(days=365)
    elif selected_timeframe == '5Y':
        start_date = current_time - timedelta(days=1825)  # Approximation of 5 years
    else:
        start_date = filtered_stock_data['Date'].min()  # Entire duration

    filtered_stock_data = filtered_stock_data[pd.to_datetime(filtered_stock_data['Date']) >= start_date]

    # Calculate RSI for the filtered data
    delta = filtered_stock_data['Closing_Price'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))

    # Create the figure for RSI chart using Plotly Express
    fig = px.line(filtered_stock_data, x=filtered_stock_data['Date'], y=rsi,
                  title=f'Relative Strength Index (RSI) for {selected_company}')
    
    # Add markers and channels for RSI visualization
    fig.add_trace(go.Scatter(
        x=filtered_stock_data['Date'],
        y=rsi,
        mode='markers',  # Markers for RSI
        marker=dict(color=rsi, colorscale='Viridis', size=10, colorbar=dict(title='RSI')),
        name='RSI Markers'
    ))

    fig.update_traces(line=dict(color='red', width=2))  # Update line color and width

    # Update layout
    fig.update_layout(
        xaxis=dict(title='Date', tickformat='%Y-%m-%d', showgrid=True, linecolor='black', linewidth=1),
        yaxis=dict(title='RSI', showgrid=True, linecolor='black', linewidth=1),
        plot_bgcolor='white',  # Set plot background color
        paper_bgcolor='white',  # Set paper (outside plot area) background color
        font=dict(family='Arial', size=12, color='Black')  # Set font style and size
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, port = 8053)

In [12]:
# Calculate Bollinger Bands
window = 20  # You can adjust the window size as needed
stock_data['Middle_Band'] = stock_data['Closing_Price'].rolling(window=window).mean()
stock_data['Upper_Band'] = stock_data['Middle_Band'] + 2 * stock_data['Closing_Price'].rolling(window=window).std()
stock_data['Lower_Band'] = stock_data['Middle_Band'] - 2 * stock_data['Closing_Price'].rolling(window=window).std()

# Create Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.Label('Select Company:'),
    dcc.Dropdown(
        id='company-dropdown',
        options=[{'label': company, 'value': company} for company in stock_data['Name'].unique()],
        value=stock_data['Name'].iloc[0]  # Default value for dropdown
    ),
    html.Br(),
    html.Label('Select Timeframe:'),
    dcc.RadioItems(
        id='timeframe-radio',
        options=[
            {'label': 'Last 1 Month', 'value': '1M'},
            {'label': 'Last 5 Months', 'value': '5M'},
            {'label': 'Last 1 Year', 'value': '1Y'},
            {'label': 'Last 5 Years', 'value': '5Y'},
            {'label': 'Entire Duration', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'block'}
    ),
    dcc.Graph(id='bollinger-bands-chart')
])

# Define callback to update the Bollinger Bands chart based on selected company and timeframe
@app.callback(
    Output('bollinger-bands-chart', 'figure'),
    [Input('company-dropdown', 'value'),
     Input('timeframe-radio', 'value')]
)
def update_chart(selected_company, selected_timeframe):
    filtered_stock_data = stock_data[stock_data['Name'] == selected_company]

    if selected_timeframe == '1M':
        start_date = current_time - timedelta(days=30)
    elif selected_timeframe == '5M':
        start_date = current_time - timedelta(days=150)
    elif selected_timeframe == '1Y':
        start_date = current_time - timedelta(days=365)
    elif selected_timeframe == '5Y':
        start_date = current_time - timedelta(days=1825)  # Approximation of 5 years
    else:
        start_date = filtered_stock_data['Date'].min()  # Entire duration

    filtered_stock_data = filtered_stock_data[pd.to_datetime(filtered_stock_data['Date']) >= start_date]

    fig = go.Figure()

    # Add Bollinger Bands lines
    fig.add_trace(go.Scatter(x=filtered_stock_data['Date'], y=filtered_stock_data['Middle_Band'], mode='lines', name='Middle_Band'))
    fig.add_trace(go.Scatter(x=filtered_stock_data['Date'], y=filtered_stock_data['Upper_Band'], mode='lines', name='Upper_Band'))
    fig.add_trace(go.Scatter(x=filtered_stock_data['Date'], y=filtered_stock_data['Lower_Band'], mode='lines', name='Lower_Band'))

    # Add marker for Closing_Price
    fig.add_trace(go.Scatter(x=filtered_stock_data['Date'], y=filtered_stock_data['Closing_Price'], mode='markers', name='Closing_Price', marker=dict(color='red', size=8)))

    fig.update_layout(
        title=f'Bollinger Bands for {selected_company}',
        xaxis=dict(title='Date', tickformat='%Y-%m-%d', showgrid=True, linecolor='black', linewidth=1),
        yaxis=dict(title='Price', showgrid=True, linecolor='black', linewidth=1),
        plot_bgcolor='white',  # Set plot background color
        paper_bgcolor='white',  # Set paper (outside plot area) background color
        font=dict(family='Arial', size=12, color='Black')  # Set font style and size
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, port = 8072)

In [14]:
# Pivot the stock_dataset to have Date as the index and companies as columns
pivot_stock_data = stock_data.pivot(index='Date', columns='Name', values='Closing_Price')

# Calculate correlation matrix
correlation_matrix = pivot_stock_data.corr()

# Calculate quartiles for correlation coefficients
coefficients = correlation_matrix.values.flatten()
lower_quantile = np.percentile(coefficients, 25)
upper_quantile = np.percentile(coefficients, 75)

# Create Dash app
app = dash.Dash(__name__)

# Define contours for lower and upper quartiles
contour_lower = go.Contour(
    z=correlation_matrix.values,
    x=correlation_matrix.index,
    y=correlation_matrix.columns,
    contours=dict(start=-1, end=lower_quantile, size=0.05),  # Adjust size as needed
    colorscale='Blues',  # Choose a colorscale for lower quartile contours
    showscale=False,
    name=f'Lower Quartile Contours (-1 - {lower_quantile:.2f})'
)

contour_upper = go.Contour(
    z=correlation_matrix.values,
    x=correlation_matrix.index,
    y=correlation_matrix.columns,
    contours=dict(start=upper_quantile, end=1, size=0.05),  # Adjust size as needed
    colorscale='Reds',  # Choose a colorscale for upper quartile contours
    showscale=False,
    name=f'Upper Quartile Contours ({upper_quantile:.2f} - 1)'
)

# Create Dash app layout
app.layout = html.Div([
    dcc.Graph(
        id='correlation-heatmap',
        figure={
            'data': [
                go.Heatmap(
                    z=correlation_matrix.values,
                    x=correlation_matrix.index,
                    y=correlation_matrix.columns,
                    colorscale='Viridis',
                    colorbar=dict(title='Correlation'),
                ),
                contour_lower,  # Add lower quartile contours
                contour_upper   # Add upper quartile contours
            ],
            'layout': {
                'title': 'Correlation Heatmap with Quartile Contours',
                'xaxis': dict(title='Companies'),
                'yaxis': dict(title='Companies'),
                'paper_bgcolor': 'white',
                'font': dict(family='Arial', size=12, color='Black')
            }
        }
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, port = 8073)